In [1]:
import json
import os
import openai
import random
import copy


client = openai.OpenAI()

In [2]:
current_question = {
    "section_name": "self-check-questions",
    "section_text": "Question: \n2. \n\n  What are the drawbacks to analyzing the global economy on a regional basis?\n  \n Answer: 2. \n\n  A region can have some of high-income countries and some of the low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capital real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.\n  \n",
    "chapter_name": "32-self-check-questions",
    "textbook_name": "principles-economics-3e",
    "chapter_url": "https://openstax.org/books/principles-economics-3e/pages/32-self-check-questions",
    "question": "\n2. \n\n  What are the drawbacks to analyzing the global economy on a regional basis?\n  \n",
    "answer": "2. \n\n  A region can have some of high-income countries and some of the low-income countries. Aggregating per capita real GDP will vary widely across countries within a region, so aggregating data for a region has little meaning. For example, if you were to compare per capital real GDP for the United States, Canada, Haiti, and Honduras, it looks much different than if you looked at the same data for North America as a whole. Thus, regional comparisons are broad-based and may not adequately capture an individual country’s economic attributes.\n  \n"
}

In [3]:
prompt_extract_features = f"""
Question: {current_question['question']}
Gold Answer: {current_question['answer']}
Your task: Think about the question and understand the answer first, then extract complete bullet points that are keyfeatures based on the answer for rebuilding the answer from scratch for the same question, more features will be helpful.
Return a JSON object in this exact format:
        {{
            "Analysis": "Analysis of the question, answer, and the features",
            "Features": [
                {{"feature": "string", "detail": "string"}},
                {{"feature": "string", "detail": "string"}},
                ... and so on, until all features are covered.
            ]
        }}
"""
temperature = 0.0

response_extract_features = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                # {"role": "system", "content": "You are an economics export and a knowledgeable teacher. Think concisely and precisely. If you meet difficulty, ask for clarification. pause and think deeply before you answer."},
                {"role": "user", "content": prompt_extract_features}
            ],
            temperature=temperature,
            # max_tokens=2000,
            response_format={"type": "json_object"}
        )

gpt_answer_extract_features = response_extract_features.choices[0].message.content
json_extract_features = json.loads(gpt_answer_extract_features)


In [4]:
# random choice 3 feature
random_features = random.sample(json_extract_features['Features'], k=3)
print(random_features)


[{'feature': 'Broad-based Analysis', 'detail': 'Regional analysis tends to be broad and may not capture specific economic characteristics of countries within the region.'}, {'feature': 'Data Aggregation', 'detail': 'Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights.'}, {'feature': 'Example of North America', 'detail': 'The example of the United States, Canada, Haiti, and Honduras illustrates how regional data can mask individual country differences.'}]


In [5]:
random_features

[{'feature': 'Broad-based Analysis',
  'detail': 'Regional analysis tends to be broad and may not capture specific economic characteristics of countries within the region.'},
 {'feature': 'Data Aggregation',
  'detail': 'Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights.'},
 {'feature': 'Example of North America',
  'detail': 'The example of the United States, Canada, Haiti, and Honduras illustrates how regional data can mask individual country differences.'}]

In [6]:
prompt_swap_feature_p1 = f"""
Task: For the selected feature, what is a misleading direction of the feature for the question. \
Modify or rewrite the feature to create a misleading feature. Keep other features the same.
Question: {current_question['question']}
"""


prompt_swap_feature_p3 = f"""
Return a JSON object in this exact format:
        {{
            "analysis_1": "How the misleading feature can be created. Think step by step.",
            "analysis_2": "Think deeper.",
            "new_feature": "string",
            "new_feature_detail": "string",
            "old_feature": "string",
            ]
        }}
"""

In [7]:
swapped_feature_list = []

count = 0
for feature in random_features:
    print(f'{count}th feature: {feature}')
    count += 1
    prompt_swap_feature_p2 = ''
    prompt_swap_feature_p2 += f"Selected Feature: {feature['feature']}. Description: {feature['detail']}\n"
    # construct the prompt with the selected feature and the other unselected features
    for other_feature in json_extract_features['Features']:
        if feature['feature'] != other_feature['feature']:
            prompt_swap_feature_p2 += f"Other Feature: {other_feature['feature']}. Description: {other_feature['detail']}\n"

    prompt_swap_feature = prompt_swap_feature_p1 + prompt_swap_feature_p2 + prompt_swap_feature_p3

    response_swap_feature = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "user", "content": prompt_swap_feature}
        ],
        temperature=temperature,
        response_format={"type": "json_object"}
    )
    gpt_answer_swap_feature = response_swap_feature.choices[0].message.content
    swapped_feature_list.append(gpt_answer_swap_feature)


0th feature: {'feature': 'Broad-based Analysis', 'detail': 'Regional analysis tends to be broad and may not capture specific economic characteristics of countries within the region.'}
1th feature: {'feature': 'Data Aggregation', 'detail': 'Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights.'}
2th feature: {'feature': 'Example of North America', 'detail': 'The example of the United States, Canada, Haiti, and Honduras illustrates how regional data can mask individual country differences.'}


In [8]:
temp = json.loads(swapped_feature_list[0])
temp


{'analysis_1': 'To create a misleading feature, we can focus on the idea that regional analysis provides a comprehensive understanding of economic trends. By emphasizing the broad-based nature of regional analysis, we can suggest that it captures all necessary economic characteristics, which can mislead one into thinking that regional analysis is sufficient for understanding the global economy.',
 'analysis_2': "By overstating the comprehensiveness of regional analysis, we ignore the nuances and specific economic conditions of individual countries. This can lead to a false sense of security in the data's accuracy and applicability, potentially resulting in misguided economic policies or strategies.",
 'new_feature': 'Comprehensive Regional Insight',
 'new_feature_detail': 'Regional analysis provides a complete and thorough understanding of economic trends, capturing all necessary economic characteristics of the countries within the region.',
 'old_feature': 'Broad-based Analysis'}

In [9]:
# for each swapped feature, find the corresponding feature in the original features, and swap the feature and detail
new_features_list  = []

for swapped_feature in swapped_feature_list:
    new_feature = json.loads(swapped_feature)
    # copy the original features
    new_features = copy.deepcopy(json_extract_features['Features'])
    for feature in new_features:
        if feature['feature'] == new_feature['old_feature']:
            feature['feature'] = new_feature['new_feature']
            feature['detail'] = new_feature['new_feature_detail']
    new_features_list.append(new_features)

new_features_list

[[{'feature': 'Economic Diversity',
   'detail': 'Regions can include both high-income and low-income countries, leading to significant variability in economic data.'},
  {'feature': 'Data Aggregation',
   'detail': 'Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights.'},
  {'feature': 'Misleading Comparisons',
   'detail': 'Regional comparisons may not accurately reflect the economic attributes of individual countries.'},
  {'feature': 'Example of North America',
   'detail': 'The example of the United States, Canada, Haiti, and Honduras illustrates how regional data can mask individual country differences.'},
  {'feature': 'Comprehensive Regional Insight',
   'detail': 'Regional analysis provides a complete and thorough understanding of economic trends, capturing all necessary economic characteristics of the countries within the region.'}],
 [{'feature': 'Economic Diversity',
   'detail': 'Regions can include both high-in

In [12]:
new_features_list[0]

[{'feature': 'Economic Diversity',
  'detail': 'Regions can include both high-income and low-income countries, leading to significant variability in economic data.'},
 {'feature': 'Data Aggregation',
  'detail': 'Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights.'},
 {'feature': 'Misleading Comparisons',
  'detail': 'Regional comparisons may not accurately reflect the economic attributes of individual countries.'},
 {'feature': 'Example of North America',
  'detail': 'The example of the United States, Canada, Haiti, and Honduras illustrates how regional data can mask individual country differences.'},
 {'feature': 'Comprehensive Regional Insight',
  'detail': 'Regional analysis provides a complete and thorough understanding of economic trends, capturing all necessary economic characteristics of the countries within the region.'}]

In [18]:
# for each new_features, construct the prompt of the question with the new features

prompt_generate_choice_p1 = f"""
Features:
"""

prompt_generate_choice_p2 = f"""
Task: Generate the answer for the given question with the given features, all of them are high level features and you need follow the directions
Question: {current_question['question']}
"""

answer_list = []
for new_features in new_features_list:
    prompt_generate_choice = ''
    for temp in new_features:
        temp_str = f"{temp['feature']}: {temp['detail']} \n"
        prompt_generate_choice += temp_str
    prompt_generate_choice = prompt_generate_choice_p1 + prompt_generate_choice
    prompt_generate_choice = prompt_generate_choice + prompt_generate_choice_p2
    
    # generate the answer
    response_generate_choice = client.chat.completions.create(
        model="gpt-4o",
        messages=[{"role": "user", "content": prompt_generate_choice}],
        temperature=temperature,
    )

    gpt_answer_generate_choice = response_generate_choice.choices[0].message.content
    print(gpt_answer_generate_choice)
    answer_list.append(gpt_answer_generate_choice)



Task: Generate the answer for the given question with the given features.
Question: 
2. 

  What are the drawbacks to analyzing the global economy on a regional basis?
  

Features:
Economic Diversity: Regions can include both high-income and low-income countries, leading to significant variability in economic data. 
Data Aggregation: Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights. 
Misleading Comparisons: Regional comparisons may not accurately reflect the economic attributes of individual countries. 
Example of North America: The example of the United States, Canada, Haiti, and Honduras illustrates how regional data can mask individual country differences. 
Comprehensive Regional Insight: Regional analysis provides a complete and thorough understanding of economic trends, capturing all necessary economic characteristics of the countries within the region. 

Analyzing the global economy on a regional basis has severa

In [21]:
print(answer_list[0])

Analyzing the global economy on a regional basis has several drawbacks:

1. **Economic Diversity**: Regions often encompass a mix of high-income and low-income countries, which leads to significant variability in economic data. This diversity can obscure the economic realities of individual countries, making it difficult to draw accurate conclusions about the region as a whole.

2. **Data Aggregation**: When economic data, such as per capita real GDP, is aggregated across countries within a region, it can result in statistics that lack meaningful insights. This aggregation can smooth over important differences and trends that are crucial for understanding the economic conditions of specific countries.

3. **Misleading Comparisons**: Regional comparisons may not accurately reflect the economic attributes of individual countries. For instance, comparing regions might lead to misleading conclusions if the economic performance of a few large or wealthy countries skews the data, overshadowi

In [15]:
new_features

[{'feature': 'Economic Diversity',
  'detail': 'Regions can include both high-income and low-income countries, leading to significant variability in economic data.'},
 {'feature': 'Data Aggregation',
  'detail': 'Aggregating per capita real GDP across countries within a region can result in data that lacks meaningful insights.'},
 {'feature': 'Misleading Comparisons',
  'detail': 'Regional comparisons may not accurately reflect the economic attributes of individual countries.'},
 {'feature': 'Regional Economic Integration',
  'detail': 'Regional analysis highlights the benefits of economic integration, such as increased trade and investment opportunities, which can overshadow the individual economic challenges faced by countries within the region.'},
 {'feature': 'Broad-based Analysis',
  'detail': 'Regional analysis tends to be broad and may not capture specific economic characteristics of countries within the region.'}]